<a href="https://colab.research.google.com/github/Adiel13/nuronal-network-2025/blob/main/Ejemplo_red_binaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Configuración e Importación de Librerías

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split


## 2. Carga y Preparación de Datos

In [2]:
try:
    data = pd.read_excel('base-de-datos-violencia-intrafamiliar-ano-2024_v3 (2).xlsx')
except FileNotFoundError:
    print("Error: Asegúrate de que la ruta del archivo Excel sea correcta.")
    # Crear un DataFrame vacío para que el código no falle si no se encuentra el archivo
    data = pd.DataFrame()


In [5]:
# Filtrar los datos, excluyendo HEC_DEPTO = 99 (Departamento Desconocido)
data2 = data[data['HEC_DEPTO'] != 99].copy()

print(data.columns.tolist())

['HEC_DIA', 'HEC_MES', 'HEC_ANO', 'HEC_DEPTO', 'HEC_DEPTOMCPIO', 'HEC_TIPAGRE', 'NUMERO_BOLETA', 'DIA_EMISION', 'MES_EMISION', 'ANO_EMISION', 'DEPTO', 'DEPTO_MCPIO', 'QUIEN_REPORTA', 'VIC_SEXO', 'VIC_EDAD', 'TOTAL_HIJOS', 'NUM_HIJ_HOM', 'NUM_HIJ_MUJ', 'VIC_ALFAB', 'VIC_ESCOLARIDAD', 'VIC_EST_CIV', 'VIC_GRUPET', 'VIC_NACIONAL', 'VIC_TRABAJA', 'VIC_OCUP', 'VIC_DEDICA', 'VIC_DISC', 'TIPO_DISCAQ', 'VIC_REL_AGR', 'OTRAS_VICTIMAS', 'VIC_OTRAS_HOM', 'VIC_OTRAS_MUJ', 'VIC_OTRAS_N_OS', 'VIC_OTRAS_N_AS', 'HEC_AREA', 'HEC_RECUR_DENUN', 'INST_DONDE_DENUNCIO', 'AGR_SEXO', 'AGR_EDAD', 'AGR_ALFAB', 'AGR_ESCOLARIDAD', 'AGR_EST_CIV', 'AGR_GURPET', 'AGR_NACIONAL', 'AGR_TRABAJA', 'AGR_OCUP', 'AGR_DEDICA', 'AGRESORES_OTROS_TOTAL', 'AGR_OTROS_HOM', 'AGR_OTRAS_MUJ', 'AGR_OTROS_N_OS', 'AGR_OTRAS_N_AS', 'INST_DENUN_HECHO', 'ORGANISMO_JURISDICCIONAL', 'CONDUCENTE', 'LEY_APLICABLE', 'ARTICULOVIF1', 'ARTICULOVIF2', 'ARTICULOVIF3', 'ARTICULOVIF4', 'ARTICULOVCM1', 'ARTICULOVCM2', 'ARTICULOVCM3', 'ARTICULOVCM4', 'A

### Definición de la Variable Binaria a Predecir

**Variable Objetivo ($y$):** ¿La víctima realizó una denuncia? (Columna `VIC_DENUNCIA`)
* **1 (Denuncia):** Códigos 1 a 7 (Diferentes tipos de denuncia).
* **0 (No Denuncia):** Códigos 0, 8, 9, 99 o NaN (No denunció, No aplica, etc.).

In [7]:

# 1. Definir las variables predictoras (X) (Se mantienen las mismas)
X = data2[['VIC_TRABAJA','VIC_ESCOLARIDAD','VIC_EST_CIV', 'AGR_ESCOLARIDAD','AGR_TRABAJA','AGR_EST_CIV']]
X = X.fillna(0) # Rellenar NaN en predictoras con 0

# 2. Limpieza y Creación de la Variable Y (Binaria)
# Usamos 'HEC_RECUR_DENUN' que es la columna de recurso/denuncia que existe en tu dataset.
data2['HEC_RECUR_DENUN'] = data2['HEC_RECUR_DENUN'].fillna(0).replace(99, 0)

data2['Y_DENUNCIA'] = data2['HEC_RECUR_DENUN'].apply(
    lambda x: 1 if x >= 1 and x <= 7 else 0 # 1 si hubo denuncia (recurso), 0 si no (código 0, etc.)
)

# 3. Definir la variable objetivo (y)
y = data2['Y_DENUNCIA'].values

# 4. Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Construcción y Entrenamiento de la Red Neuronal (Clasificación Binaria)

In [8]:
# Modelo para Clasificación Binaria
model_binario = Sequential()
# Capa Oculta 1
model_binario.add(Dense(16, input_dim=6, activation='relu'))
# Capa Oculta 2
model_binario.add(Dense(8, activation='relu'))
# Capa de Salida: 1 neurona y activación 'sigmoid' para predicción de probabilidad binaria
model_binario.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compilar el modelo: Usar 'binary_crossentropy' para la función de pérdida
model_binario.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Entrenar el modelo
history = model_binario.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8261 - loss: 0.6049 - val_accuracy: 0.9402 - val_loss: 0.2827
Epoch 2/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9352 - loss: 0.2834 - val_accuracy: 0.9366 - val_loss: 0.2479
Epoch 3/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9368 - loss: 0.2468 - val_accuracy: 0.9402 - val_loss: 0.2309
Epoch 4/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9371 - loss: 0.2307 - val_accuracy: 0.9395 - val_loss: 0.2200
Epoch 5/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9366 - loss: 0.2228 - val_accuracy: 0.9327 - val_loss: 0.2255
Epoch 6/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9362 - loss: 0.2195 - val_accuracy: 0.9315 - val_loss: 0.2233
Epoch 7/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9345 - loss: 0.2219 - val_accuracy: 0.9402 - val_loss: 0.2150
Epoch 8/50
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9328 - loss: 0.2235 - val_accuracy: 0.

In [11]:
# Evaluar el modelo
loss, accuracy = model_binario.evaluate(X_test, y_test, verbose=0)
print(f'Precisión del Modelo en Test: {accuracy:.4f}')

Precisión del Modelo en Test: 0.9402


## 4. Predicción y Explicación

### Caso de Ejemplo para Predicción

Queremos predecir la probabilidad de denuncia para una víctima (VIC) y agresor (AGR) con las siguientes características:

| Variable | Código | Descripción |
| :--- | :--- | :--- |
| **VIC_TRABAJA** | 1 | Sí trabaja |
| **VIC_ESCOLARIDAD** | 21 | Nivel Universitario (Alto) |
| **VIC_EST_CIV** | 2 | Casada/Unida |
| **AGR_ESCOLARIDAD** | 21 | Nivel Universitario (Alto) |
| **AGR_TRABAJA** | 1 | Sí trabaja |
| **AGR_EST_CIV** | 2 | Casado/Unido |

**Input:** `[1, 21, 2, 21, 1, 2]`

In [12]:
nuevo_caso = np.array([[1, 21, 2, 21, 1, 2]])
probabilidad_denuncia = model_binario.predict(nuevo_caso)

print(f'Probabilidad de Denuncia: {probabilidad_denuncia[0][0]:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Probabilidad de Denuncia: 0.9759
